# Basic Data Architecture

<img src='job_architecture.png'>

# Data Engineering - ETL

## Librairies

In [64]:
import pandas as pd
import numpy as np
import sqlite3

## Extract

In [57]:
url = 'http://app-sellaci.eu-central-1.elasticbeanstalk.com/hospital_data?collection={collection}'

df_physicians_extracted = pd.read_json(url.format(collection='physicians'))
df_consultations_extracted = pd.read_json(url.format(collection='consultations'))
df_patients_extracted = pd.read_json(url.format(collection='patients'))

print('physicians')
display(df_physicians_extracted.tail(1))
print('consultations')
display(df_consultations_extracted.tail(1))
print('patients')
display(df_patients_extracted.tail(1))

physicians


birth_date graduated_date   id           name  renowned_coefficient  \
499         NaN     2020-03-10  129  Austin Knight                    16   

    retired_date                              speciality  
499   2020-09-16  [digestive-and-visceral-surgeon, 1, 0]

consultations


device_os  duration    id  is_free_act  patient_id  \
5047  ___ ip 7WebApp 283        10  3459            0       73146   

      physician_id  price           start_date  status  
5047           303   25.0  2020-07-06 12:10:00  closed

patients


birth_date                created_date gender     id  \
93124  1970-05-01  2020-10-31 17:58:24.158000      F  40573   

                   last_vist_date  
93124  2020-10-31 19:24:43.944000

## Transform

In [58]:
df_physicians_extracted[['speciality_cleaned', 'is_doctor', 'is_mental']] = (
    pd.DataFrame(df_physicians_extracted['speciality'].tolist(), index=df_physicians_extracted.index)
)
df_physicians_extracted.tail()

birth_date graduated_date   id             name  renowned_coefficient  \
495         NaN     2020-01-10   24     John Johnson                    23   
496         NaN     2019-12-18   53  Steven Petersen                    26   
497         NaN     2020-03-27  240   Sharon Frazier                    14   
498         NaN     2020-03-20  265     Gary Flowers                     5   
499         NaN     2020-03-10  129    Austin Knight                    16   

    retired_date                              speciality  \
495         None          [occupational-therapist, 0, 1]   
496   2020-12-02                    [orthodontist, 1, 0]   
497   2020-12-02                    [orthodontist, 1, 0]   
498   2020-12-02                    [orthodontist, 1, 0]   
499   2020-09-16  [digestive-and-visceral-surgeon, 1, 0]   

                 speciality_cleaned  is_doctor  is_mental  
495          occupational-therapist          0          1  
496                    orthodontist          1          0  
497                    orthodontist          1          0  
498                    orthodontist          1          0  
499  digestive-and-visceral-surgeon          1          0

In [97]:
df_specialities_cleaned  = (
    df_physicians_extracted[['speciality_cleaned', 'is_doctor', 'is_mental']]\
        .drop_duplicates()\
        .reset_index(drop=True)\
        .rename(columns={'speciality_cleaned': "label"})
)
df_specialities_cleaned.tail()

label  is_doctor  is_mental
40                       osteopath          0          0
41                 psychotherapist          0          1
42          occupational-therapist          0          1
43                    orthodontist          1          0
44  digestive-and-visceral-surgeon          1          0

In [73]:
# remove birthdate because column is empty
df_physicians_cleaned = df_physicians_extracted[[
    'id',
    'name',
    'graduated_date',
    'renowned_coefficient',
    'retired_date',
    'speciality_cleaned',
]].rename(columns={'speciality_cleaned': "speciality"})

df_physicians_cleaned.tail()

id             name graduated_date  renowned_coefficient retired_date  \
495   24     John Johnson     2020-01-10                    23         None   
496   53  Steven Petersen     2019-12-18                    26   2020-12-02   
497  240   Sharon Frazier     2020-03-27                    14   2020-12-02   
498  265     Gary Flowers     2020-03-20                     5   2020-12-02   
499  129    Austin Knight     2020-03-10                    16   2020-09-16   

                         speciality  
495          occupational-therapist  
496                    orthodontist  
497                    orthodontist  
498                    orthodontist  
499  digestive-and-visceral-surgeon

In [66]:
df_consultations_extracted['device_os_cleaned'] = df_consultations_extracted['device_os'].apply(
    lambda x: x.split(' ')[2].replace('7', '') if type(x) == str else np.nan
)
df_consultations_extracted

device_os  duration    id  is_free_act  patient_id  \
0      ___ ip 7WebApp 269        15  2350            0       46026   
1     ___ ip 7Android 406        20  2300            0       50611   
2          ___ ip 7iOS 65        15  1277            0       44851   
3      ___ ip 7WebApp 389        60  3334            0       43742   
4                    None        15  1387            0       30469   
...                   ...       ...   ...          ...         ...   
5043  ___ ip 7Android 265        20   782            0       72508   
5044  ___ ip 7Android 189        15  1309            0       80446   
5045   ___ ip 7WebApp 281        15   774            0       29362   
5046      ___ ip 7iOS 329        15  1519            0       30034   
5047   ___ ip 7WebApp 283        10  3459            0       73146   

      physician_id  price           start_date  status device_os_cleaned  
0              289   25.0  2020-03-27 09:00:00  closed            WebApp  
1              426   25.0  2020-04-15 15:00:00  closed           Android  
2               85    0.0  2020-02-10 09:45:00  closed               iOS  
3              409   30.0  2020-02-10 19:00:00  closed            WebApp  
4              190   25.0  2020-06-26 19:00:00  closed               NaN  
...            ...    ...                  ...     ...               ...  
5043           285   30.0  2020-05-01 12:00:00  closed           Android  
5044           209   25.0  2020-03-11 12:00:00  closed           Android  
5045           301    0.0  2020-03-26 11:00:00  closed            WebApp  
5046           349   25.0  2020-07-17 09:30:00  closed               iOS  
5047           303   25.0  2020-07-06 12:10:00  closed            WebApp  

[5048 rows x 10 columns]

In [67]:
df_consultations_cleaned = df_consultations_extracted[[
    'id',
    'patient_id',
    'physician_id',
    'start_date',
    'status',
    'duration',
    'price',
    'is_free_act',
    'device_os_cleaned',
]].rename(
    columns={'device_os_cleaned': 'device_os'}
)

df_consultations_cleaned.tail()

id  patient_id  physician_id           start_date  status  duration  \
5043   782       72508           285  2020-05-01 12:00:00  closed        20   
5044  1309       80446           209  2020-03-11 12:00:00  closed        15   
5045   774       29362           301  2020-03-26 11:00:00  closed        15   
5046  1519       30034           349  2020-07-17 09:30:00  closed        15   
5047  3459       73146           303  2020-07-06 12:10:00  closed        10   

      price  is_free_act device_os  
5043   30.0            0   Android  
5044   25.0            0   Android  
5045    0.0            0    WebApp  
5046   25.0            0       iOS  
5047   25.0            0    WebApp

In [70]:
df_patients_cleaned = df_patients_extracted
df_patients_cleaned.tail()

birth_date                created_date gender     id  \
93120  1976-06-01  2020-10-31 12:42:57.271000      F  69202   
93121  1987-06-01  2020-10-31 14:44:51.580000      M  86021   
93122  1983-09-01  2020-10-31 15:05:46.311000      F  88640   
93123  2015-03-01  2020-10-31 15:07:07.453000      F  49466   
93124  1970-05-01  2020-10-31 17:58:24.158000      F  40573   

                   last_vist_date  
93120  2020-10-31 14:48:39.502000  
93121  2020-10-31 16:05:12.287000  
93122  2020-10-31 16:12:52.548000  
93123  2020-10-31 21:53:04.072000  
93124  2020-10-31 19:24:43.944000

## Load

In [99]:
conn = sqlite3.connect('hospitals_database.db')
c = conn.cursor()

c.execute(f"CREATE TABLE IF NOT EXISTS specialities ({', '.join(df_specialities_cleaned.columns)})")
conn.commit()
df_specialities_cleaned.to_sql('specialities', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS physicians ({', '.join(df_physicians_cleaned.columns)})")
conn.commit()
df_physicians_cleaned.to_sql('physicians', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS consultations ({', '.join(df_consultations_cleaned.columns)})")
conn.commit()
df_consultations_cleaned.to_sql('consultations', conn, if_exists='replace', index = False)

c.execute(f"CREATE TABLE IF NOT EXISTS patients ({', '.join(df_patients_cleaned.columns)})")
conn.commit()
df_patients_cleaned.to_sql('patients', conn, if_exists='replace', index = False)

# The Database Schema

<img src='schema.png'>

Read 1 to N.  
Ex: <i>"1 physician can do N consultations"</i>

# Analysis

In [106]:
con = sqlite3.connect('hospitals_database.db')

# SELECT
<b>Display Physicians overview</b>

In [107]:
pd.read_sql(
    '''
        SELECT 
            *
        FROM 
            physicians
    ''', 
    con
)

id                name graduated_date  renowned_coefficient  \
0     88  Claudia Washington     2017-01-03                   422   
1     78      Angela Francis     2019-01-26                   245   
2    238        Nicole Horne     2019-07-19                  1261   
3    283      Nancy Valencia     2020-01-06                    31   
4    164      Mason Campbell     2019-11-05                  3245   
..   ...                 ...            ...                   ...   
495   24        John Johnson     2020-01-10                    23   
496   53     Steven Petersen     2019-12-18                    26   
497  240      Sharon Frazier     2020-03-27                    14   
498  265        Gary Flowers     2020-03-20                     5   
499  129       Austin Knight     2020-03-10                    16   

    retired_date                      speciality  
0           None                         dentist  
1           None                         dentist  
2           None                         dentist  
3           None                         dentist  
4           None                         dentist  
..           ...                             ...  
495         None          occupational-therapist  
496   2020-12-02                    orthodontist  
497   2020-12-02                    orthodontist  
498   2020-12-02                    orthodontist  
499   2020-09-16  digestive-and-visceral-surgeon  

[500 rows x 6 columns]

<b>Display Physicians Names and Specialities</b>

In [108]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality
        FROM 
            physicians
        LIMIT 5
    ''', 
    con
)

name speciality
0  Claudia Washington    dentist
1      Angela Francis    dentist
2        Nicole Horne    dentist
3      Nancy Valencia    dentist
4      Mason Campbell    dentist

# WHERE
## Simple
<b>Display General Practician names and speciality</b>

In [109]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality
        FROM 
            physicians
        WHERE
            speciality='general-practician'
        LIMIT 5
    ''', 
    con
)

name          speciality
0        Joanne Wright  general-practician
1           Kevin Cain  general-practician
2  Kimberly Stuart DDS  general-practician
3          Kelly Smith  general-practician
4       Elizabeth Hill  general-practician

## AND
<b>Display General Practician with Renowned Coefficient above 1k names and speciality</b>

In [110]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND renowned_coefficient > 1000
        LIMIT 5
    ''', 
    con
)

name          speciality  renowned_coefficient
0  Joanne Wright  general-practician                  4392
1    Kelly Smith  general-practician                 10165
2     Shane Ward  general-practician                  1487
3    Sarah Klein  general-practician                  3983
4  Ashley Guzman  general-practician                  3521

## OR
<b>Display General Practician names and speciality with either Renowned Coefficient above 1k OR gradute date before january 2018 </b>

In [111]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND (renowned_coefficient > 1000 OR graduated_date < 2019-01-01)
        LIMIT 5
    ''', 
    con
)

name          speciality  renowned_coefficient
0  Joanne Wright  general-practician                  4392
1    Kelly Smith  general-practician                 10165
2     Shane Ward  general-practician                  1487
3    Sarah Klein  general-practician                  3983
4  Ashley Guzman  general-practician                  3521

## LIKE
<b>Display General Practician with name starting by "Cha"</b>

In [112]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND name LIKE 'Cha%'
        LIMIT 5
    ''', 
    con
)

name          speciality  renowned_coefficient
0  Charles Brown  general-practician                   343

## BETWEEN
<b>Display Physicians graduated in March 2019</b>

In [113]:
pd.read_sql(
    '''
        SELECT 
            name, 
            graduated_date
        FROM 
            physicians
        WHERE
            graduated_date BETWEEN '2020-03-01' AND '2020-04-01'
        LIMIT 5
    ''', 
    con
)

name graduated_date
0  Devin Bautista     2020-03-27
1  Richard Barker     2020-03-24
2       Frank Cox     2020-03-18
3   Russell Welch     2020-03-24
4   Dale Gonzalez     2020-03-13

## NULL
<b>Display Physicians still active</b>

In [114]:
pd.read_sql(
    '''
        SELECT 
            name, 
            graduated_date
        FROM 
            physicians
        WHERE
            retired_date IS NULL
        LIMIT 5
    ''', 
    con
)

name graduated_date
0  Claudia Washington     2017-01-03
1      Angela Francis     2019-01-26
2        Nicole Horne     2019-07-19
3      Nancy Valencia     2020-01-06
4      Mason Campbell     2019-11-05

# ORDER
<b>Display 2 highest Renowned coefficient GP (general practician)</b>

In [115]:
pd.read_sql(
    '''
        SELECT 
            name, 
            speciality,
            renowned_coefficient
        FROM 
            physicians
        WHERE
            speciality='general-practician'
            AND renowned_coefficient > 1000
        ORDER BY renowned_coefficient DESC
        LIMIT 2
    ''', 
    con
)

name          speciality  renowned_coefficient
0    Melissa Baldwin  general-practician                 23627
1  Geoffrey Phillips  general-practician                 16543

# AGGREGATING FUNCTIONS
## COUNT / MIN / MAX / AVG
<b>Get number of physicians in physician table</b>  
<b>Get older physician, last retired and average renowned in physician table</b>

In [116]:
pd.read_sql(
    '''
        SELECT 
            COUNT(*) AS nb_rows_physicians,
            MAX(retired_date) AS last_retired_date,
            AVG(renowned_coefficient) AS avg_renowned
        FROM 
            physicians
    ''', 
    con
)

nb_rows_physicians last_retired_date  avg_renowned
0                 500        2021-08-11      1172.698

And so many other aggregation functions !

# GROUP BY 
## COUNT
<b>Get number of physicians not retired by speciality</b>

In [117]:
pd.read_sql(
    '''
        SELECT 
            speciality,
            count(*) AS nb_physicians
        FROM 
            physicians
        WHERE
            retired_date IS NULL
        GROUP BY
            speciality
        ORDER BY 
            nb_physicians DESC
        LIMIT 5
    ''', 
    con
)

speciality  nb_physicians
0  general-practician            156
1        psychologist             31
2           dietitian             24
3  physical-therapist             22
4        pediatrician             11

# HAVING
<b>Then where after Group By !</b>
<b>Get number of physicians not retired by speciality with more than 25 physicians</b>

In [118]:
pd.read_sql(
    '''
        SELECT 
            speciality,
            count(*) AS nb_physicians
        FROM 
            physicians
        WHERE
            retired_date IS NULL
        GROUP BY
            speciality
        HAVING 
            nb_physicians > 25
        ORDER BY 
            nb_physicians DESC
        LIMIT 5
    ''', 
    con
)

speciality  nb_physicians
0  general-practician            156
1        psychologist             31

# JOIN 

<img src='joins.png'>

## INNER JOIN
<b>Get number of consultations made by physicians who consulted</b>

In [119]:
pd.read_sql(
    '''
        SELECT 
            p.name,
            count(*) AS nb_consultations
        FROM 
            physicians p
        INNER JOIN 
            consultations c ON p.id = c.physician_id
        GROUP BY
            p.id, p.name
        ORDER BY 
            nb_consultations DESC 
        LIMIT 5
    ''', 
    con
)

name  nb_consultations
0  Melissa Baldwin               170
1      Kayla Stone               150
2     Donna Walker               135
3    William Salas               133
4      Karen Stone               133

## LEFT JOIN
<b>Get number of consultations made by physicians</b>

In [120]:
pd.read_sql(
    '''
        SELECT 
            p.id, 
            COUNT(c.id) as nb_consultations
        FROM 
            physicians p
        LEFT JOIN 
            consultations c ON c.physician_id = p.id
        GROUP BY 
            p.id
        ORDER BY 
            nb_consultations DESC
        LIMIT 5
    ''', 
    con
)

id  nb_consultations
0   31               170
1   84               150
2  349               135
3  303               133
4   86               133

## UNION

In [122]:
pd.read_sql(
    '''
        SELECT 
            'mental' AS speciality,
            COUNT(*) AS nb_consultations
        FROM 
            consultations c 
            INNER JOIN physicians p ON c.physician_id = p.id
            INNER JOIN specialities s ON s.label = p.speciality
            WHERE s.is_mental = True
            
        UNION
        
        SELECT 
            'others' AS speciality,
            COUNT(*) AS nb_consultations
        FROM 
            consultations c 
            INNER JOIN physicians p ON c.physician_id = p.id
            INNER JOIN specialities s ON s.label = p.speciality
            WHERE s.is_mental = False    
    ''', 
    con
)

speciality  nb_consultations
0     mental               157
1     others              4891

Check CROSS JOIN & OUTER JOIN on your side

# CTE - Common Table Expression

In [123]:
pd.read_sql(
    '''
        WITH mental_consultations AS (
            SELECT 
                'mental' AS speciality,
                COUNT(*) AS nb_consultations
            FROM 
                consultations c 
                INNER JOIN physicians p ON c.physician_id = p.id
                INNER JOIN specialities s ON s.label = p.speciality
                WHERE s.is_mental = True
        ),
                    
        physical_consultations AS (
            SELECT 
                'others' AS speciality,
                COUNT(*) AS nb_consultations
            FROM 
                consultations c 
                INNER JOIN physicians p ON c.physician_id = p.id
                INNER JOIN specialities s ON s.label = p.speciality
                WHERE s.is_mental = False    
        )
        
        SELECT 
            *
        FROM
            mental_consultations 
        
        UNION
        
        SELECT 
            *
        FROM 
            physical_consultations
    ''', 
    con
)

speciality  nb_consultations
0     mental               157
1     others              4891

# Business Analyst Question

<b>I'm the hospital Director. Give me 10 KPIs I should follow every week.</b>  
<b>I'm the hospital Data Engineer. Give me 5 KPIs I should follow to check my data veracity.</b>  

# SQL Queries

## Consultations
<b>Get number of consultations by week</b>    
<b>Get number of consultations by month for GP (general practicians)</b> 

## Physicians 
<b>Number of new physicians by week</b> 
<b>Number of retired physicians by week</b> 

## Patients
<b>Number of new patients who are in the hospital database by week</b>  
<b>Number of patients who consulted by week</b>  
<b>Number of new patients who consulted by week</b>  
<b>Number of patients who came back for a consultation by week</b>  

## Others
<b>Get price earn by the hospital by week</b>   
<b>Get raito #price earned / #consultations made</b>  
<b>Get most paid physician by month</b>  
<b>Get physician having TC the most by week</b>  
<b>Get most prorfitable speciality by month</b>   
<b>Get top 10 medication ever</b>   
<b>Get physician who have the highet prescription/consultation ratio</b>   
<b>How many consultation did physicians who retired ?</b>   
<b>Should we call back a physican ? If yes, who ?</b>   
<b>How long a physian stay in the hospital in average ?</b>   

## Usage
<b>Which device is the most used for booking consultations ?</b>  
<b>Do we have physicians who never consulted ?</b>  
<b>Physician doing too many free act ?</b>   
<b>How many times the best physican/patient couple consulted togeter ?</b>   
<b>What is the median age difference between a physician and its patient ?</b>   